# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [61]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [15]:
file = os.path.join("output_data","raw_data.csv")
csv_df = pd.read_csv(file)
csv_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,hobart,-42.88,147.33,46.40,61,20,18.34,AU,1600029790
1,port elizabeth,-33.92,25.57,55.40,76,0,10.29,ZA,1600029790
2,isangel,-19.55,169.27,72.95,78,61,15.21,VU,1600029790
3,faanui,-16.48,-151.75,78.26,79,91,11.56,PF,1600029791
4,souillac,-20.52,57.52,71.60,78,75,10.29,MU,1600029791


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [154]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Create mapping figure
fig = gmaps.figure()

# Pull data from DF.
locations = csv_df[['Lat','Lng']]
humidity = csv_df['Humidity']

# create the heatmap layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, max_intensity = humidity.max() ,point_radius=10)

# add the heatmap layer to the figure
fig.add_layer(heat_layer)

# display the figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows with null values.

In [58]:
# Ideal temps between 32-85 def F, with humidity of <50. 
ideal_cities_df = csv_df.loc[(csv_df['Max Temp'] <= 85) & (csv_df['Max Temp'] >= 60) & (csv_df['Humidity'] < 35)]
ideal_cities_df.dropna(inplace=True)
len(ideal_cities_df['City'])

C:\Users\Sam\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


18

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [156]:
hotel_df = ideal_cities_df
locations = hotel_df[['Lat','Lng']]

In [157]:
# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

In [158]:
responses = []
lat_lst = []
lng_lst = []

print('Beginning Data Retrival')
print('-----------------------')
for index, row in locations.iterrows():
    try:
        params = {"location": f"{row['Lat']},{row['Lng']}",
                  "keyword": 'hotels',
                  "radius": 5000,
                  "type": 'lodging',
                  "key": g_key}

        response_json = requests.get(base_url, params=params).json()
        responses.append(response_json['results'][0]['name'])
#         lat_lst.append(response_json['results'][0]['geometry']['location']['lat'])
#         lng_lst.append(response_json['results'][0]['geometry']['location']['lng'])
        
        print(f"Processing Record for {index} | Hotel Name Result: {response_json['results'][0]['name']}")
    
    except IndexError:
        responses.append("null")
#         lat_lst.append(response_json['results'][0]['geometry']['location']['lat'])
#         lng_lst.append(response_json['results'][0]['geometry']['location']['lng'])
    print("...No hotels found... ")

hotel_df['Hotel Name'] = responses

Beginning Data Retrival
-----------------------
Processing Record for 18 | Hotel Name Result: Buffalo Bill (Cabins) Village
...No hotels found... 
Processing Record for 123 | Hotel Name Result: Hotel Barrocos
...No hotels found... 
Processing Record for 141 | Hotel Name Result: The Gerald Apartment Hotel
...No hotels found... 
Processing Record for 149 | Hotel Name Result: Residence Inn by Marriott Casper
...No hotels found... 
Processing Record for 153 | Hotel Name Result: 7 Days Premium
...No hotels found... 
Processing Record for 179 | Hotel Name Result: Gostinitsa
...No hotels found... 
Processing Record for 182 | Hotel Name Result: Tella Hotel
...No hotels found... 
Processing Record for 188 | Hotel Name Result: The Golden Hotel, Ascend Hotel Collection
...No hotels found... 
Processing Record for 193 | Hotel Name Result: Al-Dawood Hotel & Restaurant
...No hotels found... 
Processing Record for 220 | Hotel Name Result: Best Western Plus Winslow Inn
...No hotels found... 
Processin

In [140]:
# Remove hotel names with null values
hotel_df_df = hotel_df.loc[hotel_df['Hotel Name'] != 'null']
hotel_df_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
18,cody,44.53,-109.06,84.00,12,1,11.41,US,1600029794,Holiday Inn Cody-at Buffalo Bill Village
123,goiatuba,-18.01,-49.35,82.51,20,0,8.19,BR,1600029816,Hotel Baeza
141,geraldton,-28.77,114.60,64.40,34,0,16.11,AU,1600029821,The Gerald Apartment Hotel
149,casper,42.87,-106.31,82.00,18,1,16.11,US,1600029781,Residence Inn by Marriott Casper
153,hami,42.80,93.45,66.38,21,100,5.55,CN,1600029824,7 Days Premium
179,rassvet,46.83,40.75,60.80,32,0,7.74,RU,1600029831,Gostinitsa
182,viransehir,37.24,39.76,83.28,28,0,12.97,TR,1600029832,Tella Hotel
188,golden,39.76,-105.22,84.99,14,20,9.17,US,1600029833,"The Golden Hotel, Ascend Hotel Collection"
193,dalbandin,28.89,64.41,76.24,17,1,3.53,PK,1600029834,Al-Dawood Hotel & Restaurant
220,winslow,35.02,-110.70,84.20,16,1,5.82,US,1600029841,Best Western Plus Winslow Inn


In [159]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [160]:
# Create a marker layer using our coordinates
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)

# Add the layer to the map
fig.add_layer(markers)

# display the figure with the newly added layer
fig

Figure(layout=FigureLayout(height='420px'))